# Импорт библиотек


In [7]:
import os
import warnings
import logging
import shutil
import json
from pathlib import Path
from ultralytics import YOLO
import torch



# Настройки


In [8]:
warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO)



# Константы


In [9]:
CLASSES = {0: 'fire', 1: 'smoke'}

def fix_path_separators(path):
    """Исправление разделителей путей для Windows"""
    return path.replace('/', os.sep).replace('\\', os.sep)



# 1. Подготовка данных


In [10]:
def prepare_dataset():
    try:
        # Очистка предыдущих данных
        if os.path.exists('yolo_dataset'):
            shutil.rmtree('yolo_dataset')
        
        # Создание структуры папок
        os.makedirs('yolo_dataset/images/train', exist_ok=True)
        os.makedirs('yolo_dataset/images/val', exist_ok=True)
        os.makedirs('yolo_dataset/labels/train', exist_ok=True)
        os.makedirs('yolo_dataset/labels/val', exist_ok=True)

        def convert_coco_to_yolo(json_path, img_dir, out_img_dir, out_label_dir):
            with open(json_path) as f:
                data = json.load(f)
            
            # Создаем mapping для изображений
            images = {img['id']: img for img in data['images']}
            
            # Обрабатываем аннотации
            for ann in data['annotations']:
                if ann['category_id'] not in CLASSES:
                    continue
                
                img_info = images.get(ann['image_id'])
                if not img_info:
                    continue
                
                # Исправление пути к изображению
                img_name = fix_path_separators(img_info['file_name'])
                base_name = os.path.basename(img_name)
                
                # Конвертация bbox
                x, y, w, h = ann['bbox']
                img_w, img_h = img_info['width'], img_info['height']
                x_center = (x + w/2) / img_w
                y_center = (y + h/2) / img_h
                width = w / img_w
                height = h / img_h
                
                # Сохранение аннотации
                label_path = os.path.join(out_label_dir, os.path.splitext(base_name)[0] + '.txt')
                with open(label_path, 'a') as f:
                    f.write(f"{ann['category_id']} {x_center} {y_center} {width} {height}\n")
                
                # Копирование изображения с проверкой пути
                src = os.path.join(img_dir, img_name)
                dst = os.path.join(out_img_dir, base_name)
                if os.path.exists(src) and not os.path.exists(dst):
                    shutil.copy(src, dst)

        # Проверка существования исходных данных
        if not os.path.exists('475_fire_train/annotations/instances_default.json'):
            raise FileNotFoundError("Train annotations not found!")
        if not os.path.exists('474_fire_val/annotations/instances_default.json'):
            raise FileNotFoundError("Val annotations not found!")

        # Конвертация данных
        convert_coco_to_yolo(
            '475_fire_train/annotations/instances_default.json',
            '475_fire_train/images',
            'yolo_dataset/images/train',
            'yolo_dataset/labels/train'
        )
        
        convert_coco_to_yolo(
            '474_fire_val/annotations/instances_default.json',
            '474_fire_val/images',
            'yolo_dataset/images/val',
            'yolo_dataset/labels/val'
        )

        # Создание YAML файла
        yaml_content = f"""
path: {Path('yolo_dataset').absolute()}
train: images/train
val: images/val
names: {list(CLASSES.values())}
"""
        with open('fire_smoke.yaml', 'w') as f:
            f.write(yaml_content.strip())
        
        # Проверка созданных данных
        print(f"Train images: {len(os.listdir('yolo_dataset/images/train'))}")
        print(f"Train labels: {len(os.listdir('yolo_dataset/labels/train'))}")
        print(f"Val images: {len(os.listdir('yolo_dataset/images/val'))}")
        print(f"Val labels: {len(os.listdir('yolo_dataset/labels/val'))}")
        
        return True
    
    except Exception as e:
        print(f"Error preparing dataset: {str(e)}")
        return False



# 2. Обучение модели


In [11]:
def train_model():
    try:
        # Проверка YAML файла
        if not os.path.exists('fire_smoke.yaml'):
            raise FileNotFoundError("YAML config not found!")
        
        # Инициализация модели (используем детекцию вместо сегментации)
        model = YOLO('yolov8s.pt')
        
        # Параметры обучения
        train_args = {
            'data': 'fire_smoke.yaml',
            'epochs': 6,
            'batch': 4,
            'imgsz': 416,
            'device': '0' if torch.cuda.is_available() else 'cpu',
            'workers': 1,
            'verbose': True
        }
        
        # Обучение
        results = model.train(**train_args)
        return model
    
    except Exception as e:
        print(f"Training error: {str(e)}")
        return None
# больше 7 эпох не вывозит мой комп =)


# Основной процесс


In [12]:
if __name__ == "__main__":
    print("=== Starting ===")
    
    # 1. Подготовка данных
    if not prepare_dataset():
        print("Failed to prepare dataset!")
        exit()
    
    # 2. Обучение модели
    model = train_model()
    if model is None:
        print("Failed to train model!")
        exit()
    
    # 3. Сохранение модели
    model.save('best_fire_smoke.pt')
    print("\nTraining completed! Model saved as 'best_fire_smoke.pt'")

=== Starting ===
Train images: 40
Train labels: 40
Val images: 5
Val labels: 5


100%|██████████| 21.5M/21.5M [00:00<00:00, 24.0MB/s]


New https://pypi.org/project/ultralytics/8.3.107 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.89  Python-3.12.8 torch-2.6.0+cpu CPU (AMD Ryzen 7 6800HS Creator Edition)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=fire_smoke.yaml, epochs=6, time=None, patience=100, batch=4, imgsz=416, save=True, save_period=-1, cache=False, device=cpu, workers=1, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fals

train: Scanning C:\Users\Aleksandr\Downloads\Detect_fire\yolo_dataset\labels\train... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<00:00, 1420.22it/s]

train: New cache created: C:\Users\Aleksandr\Downloads\Detect_fire\yolo_dataset\labels\train.cache


...

        6/6         0G      1.547      1.449      1.255         13        416: 100%|██████████| 10/10 [00:13<00:00,  1.33s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.91it/s]

                   all          5          8      0.852      0.725      0.899      0.522



6 epochs completed in 0.023 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 22.5MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.89  Python-3.12.8 torch-2.6.0+cpu CPU (AMD Ryzen 7 6800HS Creator Edition)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.97it/s]


                   all          5          8      0.756      0.775      0.909      0.539
                 smoke          5          8      0.756      0.775      0.909      0.539
Speed: 0.3ms preprocess, 93.5ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\detect\train2

Training completed! Model saved as 'best_fire_smoke.pt'
